# Task 1 - Logistic Regression
## Team Skynet

In [37]:
import pandas as pd
import numpy as np

In [38]:
training_df = pd.read_csv('./train_tfidf_features.csv')
testing_df = pd.read_csv('./test_tfidf_features.csv')

In [39]:
training_df.dropna()
testing_df.dropna()

,id,0,1,2,3,4,5,6,7,8,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,17185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,21476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4292,21477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4293,21478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4294,21479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
feature_names = [str(i) for i in range(0,5000)]
train_features = training_df.loc[:, feature_names]
training_labels = training_df.loc[:, 'label']
testing_features = testing_df.loc[:, feature_names]
X_train, X_test = train_features.to_numpy(), testing_features.to_numpy()
Y_train = training_labels.to_numpy()

In [41]:
def normalize(X):
    m, n = X.shape
    for i in range(n):
        X = (X - X.mean(axis=0))/X.std(axis=0)
        
    return X

def sigmoid(z):
    return 1/(1+np.exp(-z))

def loss(y, y_hat):
    return -np.mean(y*(np.log(y_hat))+(1-y)*(np.log(1-y_hat)))

def gradients(X, y, y_hat):
    m = X.shape[0]
    dw = (1/m)*np.matmul(X.T, (y_hat - y))
    db = (1/m)*np.sum((y_hat - y)) 
    return dw, db

def train(X, y, bs, lr, epochs):
    m, n = X.shape

    w = np.zeros((n,1))
    b = 0
    
    
    y = y.reshape(m,1)
    
    
    #x = normalize(X)
    
    
    losses = []
    
    
    for epoch in range(epochs):
        for i in range((m-1)//bs + 1):
            
            
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            
            y_hat = sigmoid(np.matmul(xb, w) + b)
            
            
            dw, db = gradients(xb, yb, y_hat)
            
            
            w -= lr*dw
            b -= lr*db
        
        
        l = loss(y, sigmoid(np.matmul(X, w) + b))
        losses.append(l)
        
    return w, b, losses

def predict(X, w, b):
    #x = normalize(X)
    
    preds = sigmoid(np.matmul(X, w) + b)
    
    pred_class = []
    
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)


In [45]:
w_train, b_train, training_losses = train(X_train, Y_train, 32, 0.01, 1000)
Y_pred = predict(X_test, w_train, b_train)

In [46]:
skl_lr = pd.read_csv("sklearn_logistic_regression_predictions")
skl_lr_true = skl_lr['label']
from sklearn.metrics import accuracy_score
print(accuracy_score(skl_lr_true, Y_pred))

0.9464618249534451


In [47]:
Y_pred = pd.DataFrame(Y_pred)
Y_pred.columns = ['label']
Y_pred.insert(0, 'id', testing_df['id'])
Y_pred.to_csv('skynet_task1_lr_predictions', index=False)